In [ ]:
import os
import respiratory_extraction.utils as utils

data_path = os.path.join(os.getcwd(), '..', 'data', 'subjects')
dataset = utils.Dataset(data_path)

subject = 'Proband16'
scenario = '101_natural_lighting'

frames, params = dataset.read_video_bgr(subject, scenario)

In [ ]:
hyperparameters = {
    'OFP_maxCorners': 100,
    'OFP_qualityLevel': 0.1,
    'OFP_minDistance': 7,
    'OFP_mask': None,
    'OFP_QualityLevelRV': 0.05,
    'OFP_winSize': (15, 15),
    'OFP_maxLevel': 2,
    'FSS_switch': False,
    'FSS_maxCorners': 100,
    'FSS_qualityLevel': 0.1,
    'FSS_minDistance': 7,
    'FSS_mask': None,
    'FSS_QualityLevelRV': 0.05,
    'FSS_FPN': 5,
    'CGOF_switch': False,
    'Filter_switch': False,
    'Filter_type': 'bandpass',
    'Filter_order': 3,
    'Filter_LowPass': 2,
    'Filter_HighPass': 40,
    'Normalization_switch': False,
    'RR_switch': False,
    'RR_Algorithm_PC_Height': None,
    'RR_Algorithm_PC_Threshold': None,
    'RR_Algorithm_PC_MaxRR': 45,
    'RR_Algorithm_CP_shfit_distance': 15,
    'RR_Algorithm_NFCP_shfit_distance': 15,
    'RR_Algorithm_NFCP_qualityLevel': 0.6
}

In [ ]:
import numpy as np
from scipy.fftpack import fft
from scipy.signal import find_peaks


class RRAlgorithm:
    def __init__(self, respiratory_signal: np.ndarray, fs: float):
        self.data = respiratory_signal
        self.fs = fs
        self.N = len(respiratory_signal)
        self.Time = self.N / fs

    def fft(self) -> float:
        """
        Fast Fourier Transform
        :return:
        """

        fft_y = fft(self.data)
        abs_y = np.abs(fft_y)
        normalization_y = abs_y / self.N
        normalization_half_y = normalization_y[range(int(self.N / 2))]
        sorted_indices = np.argsort(normalization_half_y)

        max_frequency = self.fs
        f = np.linspace(0, max_frequency, self.N)
        return f[sorted_indices[-2]]

    def peak_counting(self,
                      height=hyperparameters['RR_Algorithm_PC_Height'],
                      threshold=hyperparameters['RR_Algorithm_PC_Threshold'],
                      max_rr=hyperparameters['RR_Algorithm_PC_MaxRR']
                      ) -> float:
        """
        Peak Counting Method
        :param height:
        :param threshold:
        :param max_rr:
        :return:
        """

        distance = 60 / max_rr * self.fs

        peaks, _ = find_peaks(
            self.data,
            height=height,
            threshold=threshold,
            distance=distance)

        return len(peaks) / self.Time

    def crossing_point(self) -> float:
        """
        Crossing Point Method
        :return:
        """

        shift_distance = int(self.fs / 2)
        data_shift = np.zeros(self.data.shape) - 1
        data_shift[shift_distance:] = self.data[:-shift_distance]
        cross_curve = self.data - data_shift

        zero_number = 0
        zero_index = []
        for inx in range(len(cross_curve) - 1):
            if cross_curve[inx] == 0:
                zero_number += 1
                zero_index.append(inx)
            else:
                if cross_curve[inx] * cross_curve[inx + 1] < 0:
                    zero_number += 1
                    zero_index.append(inx)

        return (zero_number / 2) / (self.N / self.fs)

    def negative_feedback_crossover_point_method(
            self,
            quality_level=hyperparameters['RR_Algorithm_NFCP_qualityLevel']
    ) -> float:
        shift_distance = int(self.fs / 2)
        data_shift = np.zeros(self.data.shape) - 1
        data_shift[shift_distance:] = self.data[:-shift_distance]
        cross_curve = self.data - data_shift

        zero_number = 0
        zero_index = []
        for i in range(len(cross_curve) - 1):
            if cross_curve[i] == 0:
                zero_number += 1
                zero_index.append(i)
            else:
                if cross_curve[i] * cross_curve[i + 1] < 0:
                    zero_number += 1
                    zero_index.append(i)

        rr_tmp = ((zero_number / 2) / (self.N / self.fs))

        if len(zero_index) <= 1:
            return rr_tmp

        time_span = 60 / rr_tmp / 2 * self.fs * quality_level
        zero_span = []
        for i in range(len(zero_index) - 1):
            zero_span.append(zero_index[i + 1] - zero_index[i])

        while min(zero_span) < time_span:
            doubt_point = np.argmin(zero_span)
            zero_index.pop(doubt_point)
            zero_index.pop(doubt_point)
            if len(zero_index) <= 1:
                break
            zero_span = []
            for i in range(len(zero_index) - 1):
                zero_span.append(zero_index[i + 1] - zero_index[i])

        return (zero_number / 2) / (self.N / self.fs)

In [ ]:
from scipy import signal


# Formally called FeaturePointSelectionStrategy...
def special_feature_point_selection(frame, fpn=5, quality_level=0.3):
    feature_params = {
        'maxCorners': hyperparameters['FSS_maxCorners'],
        'qualityLevel': quality_level,
        'minDistance': hyperparameters['FSS_minDistance']
    }
    points = cv2.goodFeaturesToTrack(
        frame,
        mask=hyperparameters['FSS_mask'],
        **feature_params)

    while points is None:
        feature_params['qualityLevel'] = quality_level - hyperparameters['FSS_QualityLevelRV']
        points = cv2.goodFeaturesToTrack(frame, mask=None, **feature_params)

    if len(points) < fpn:
        fpn = len(points)

    h = frame.shape[0] / 2
    w = frame.shape[1] / 2

    # TODO: Figure out how the top points are selected...
    p1 = points.copy()
    p1[:, :, 0] -= w
    p1[:, :, 1] -= h
    p1_1 = np.multiply(p1, p1)
    p1_2 = np.sum(p1_1, 2)
    p1_3 = np.sqrt(p1_2)
    p1_4 = p1_3[:, 0]
    p1_5 = np.argsort(p1_4)

    fp_map = np.zeros((fpn, 1, 2), dtype=np.float32)
    for inx in range(fpn):
        fp_map[inx, :, :] = points[p1_5[inx], :, :]

    return fp_map


def default_feature_point_selection(frame, quality_level=0.3):
    feature_params = {
        'maxCorners': hyperparameters['OFP_maxCorners'],
        'qualityLevel': quality_level,
        'minDistance': hyperparameters['OFP_minDistance']
    }
    points = cv2.goodFeaturesToTrack(frame, mask=hyperparameters['OFP_mask'], **feature_params)

    while points is None:
        feature_params['qualityLevel'] = quality_level - hyperparameters['OFP_QualityLevelRV']
        points = cv2.goodFeaturesToTrack(
            frame,
            mask=hyperparameters['OFP_mask'],
            **feature_params)

    return points

In [ ]:
# TODO: Figure out what this does...
def correlation_guided_optical_flow_method(point_amplitudes: np.ndarray, respiratory_signal: np.ndarray) -> np.ndarray:
    point_amplitudes_t = np.array(point_amplitudes).T

    augmented_matrix = np.zeros((point_amplitudes_t.shape[0] + 1, point_amplitudes_t.shape[1]))
    augmented_matrix[0, :] = respiratory_signal
    augmented_matrix[1:, :] = point_amplitudes_t

    correlation_matrix = np.corrcoef(augmented_matrix)

    cm_mean = np.mean(abs(correlation_matrix[0, 1:]))

    quality_num = np.array(abs(correlation_matrix[0, 1:]) >= cm_mean).sum()
    quality_feature_point_arg = np.array(abs(correlation_matrix[0, 1:]) >= cm_mean).argsort()[0 - quality_num:]

    cgof_matrix = np.zeros((point_amplitudes.shape[0], quality_num))

    for idx in range(quality_num):
        cgof_matrix[:, idx] = point_amplitudes[:, quality_feature_point_arg[idx]]

    return np.sum(cgof_matrix, 1) / quality_num

In [ ]:
def extract_feature_point_movement(
        frames: np.ndarray,
        quality_level=float(0.3),
        use_fft=False,
) -> np.ndarray:
    current_frame = cv2.cvtColor(frames[0], cv2.COLOR_BGR2GRAY)

    # FeaturePoint Selection Strategy
    if use_fft:
        feature_points = special_feature_point_selection(
            current_frame,
            fpn=hyperparameters['FSS_FPN'],
            quality_level=hyperparameters['FSS_qualityLevel']
        )
    else:
        feature_points = default_feature_point_selection(
            current_frame,
            quality_level=quality_level
        )

    lk_params = {
        'winSize': hyperparameters['OFP_winSize'],
        'maxLevel': hyperparameters['OFP_maxLevel'],
    }
    total_frame = len(frames)

    # Store the feature points for each frame
    feature_point_matrix = np.zeros((int(total_frame), feature_points.shape[0], 2))

    # Store the feature points for the first frame
    feature_point_matrix[0, :, 0] = feature_points[:, 0, 0].T
    feature_point_matrix[0, :, 1] = feature_points[:, 0, 1].T

    # Calculate the optical flow of the feature points for each frame
    for inx, frame in enumerate(frames[1:], start=1):
        next_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        new_positions, _, _ = cv2.calcOpticalFlowPyrLK(
            current_frame,
            next_frame,
            feature_points,
            None,
            **lk_params)

        current_frame = next_frame.copy()

        feature_points = new_positions.reshape(-1, 1, 2)
        feature_point_matrix[inx, :, 0] = feature_points[:, 0, 0].T
        feature_point_matrix[inx, :, 1] = feature_points[:, 0, 1].T

    return feature_point_matrix


def extract_respiratory_signal(
        frames: np.ndarray,
        fps: int,
        quality_level=float(0.3),
        use_fft=False,
        use_cgof=False,
        use_filter=True,
        use_normalization=False,
) -> np.ndarray:
    # Store the feature points for each frame
    feature_point_matrix = extract_feature_point_movement(
        frames,
        quality_level=quality_level,
        use_fft=use_fft,
    )

    # Calculate the amplitude of the feature points for each frame
    point_amplitudes = np.sqrt(feature_point_matrix[:, :, 0] ** 2 + feature_point_matrix[:, :, 1] ** 2)

    # Calculate the amplitude of the feature points for each frame
    respiratory_signal = np.sum(point_amplitudes, 1) / point_amplitudes.shape[1]

    # Correlation-Guided Optical Flow Method
    if use_cgof:
        respiratory_signal = correlation_guided_optical_flow_method(point_amplitudes, respiratory_signal)

    if use_filter:
        original_signal = respiratory_signal
        filter_order = hyperparameters['Filter_order']
        lowpass = hyperparameters['Filter_LowPass'] / 60
        highpass = hyperparameters['Filter_HighPass'] / 60
        b, a = signal.butter(filter_order, [2 * lowpass / fps, 2 * highpass / fps], hyperparameters['Filter_type'])
        respiratory_signal_filtered = signal.filtfilt(b, a, original_signal)
    else:
        respiratory_signal_filtered = respiratory_signal

    if use_normalization:
        max_ampl = max(respiratory_signal_filtered)
        min_ampl = min(respiratory_signal_filtered)
        respiratory_signal_norm = (respiratory_signal_filtered - min_ampl) / (max_ampl - min_ampl) - 0.5
    else:
        respiratory_signal_norm = respiratory_signal_filtered

    return respiratory_signal_norm

In [ ]:
import cv2

breathing_signal = extract_respiratory_signal(
    frames,
    params.fps,
    quality_level=hyperparameters['OFP_qualityLevel'],
    use_fft=True,
    use_cgof=True,
    use_filter=True,
    use_normalization=True,
)

In [ ]:
import plotly.express as px

fig = px.line(y=breathing_signal, title='Respiratory Signal')
fig.show()

In [ ]:
# RR Evaluation
RR_method = RRAlgorithm(breathing_signal, params.fps)
RR_FFT = RR_method.fft()
RR_PC = RR_method.peak_counting()
RR_CP = RR_method.crossing_point()
RR_NFCP = RR_method.negative_feedback_crossover_point_method()

get_ground_truth_rr = dataset.get_ground_truth_rr(subject, scenario)

print(f'Ground Truth: {round(get_ground_truth_rr * 60, 1)}')
print(f'FFT: {round(RR_FFT * 60, 1)}')
print(f'Peak Counting: {round(RR_PC * 60, 1)}')
print(f'Crossing Point: {round(RR_CP * 60, 1)}')
print(f'Negative Feedback Crossover Point: {round(RR_NFCP * 60, 1)}')

In [ ]:
import matplotlib.pyplot as plt

old_gray = cv2.cvtColor(frames[0], cv2.COLOR_BGR2GRAY)

default_points = default_feature_point_selection(
    old_gray,
    quality_level=hyperparameters['FSS_qualityLevel'])

special_points = special_feature_point_selection(
    old_gray,
    fpn=hyperparameters['FSS_FPN'],
    quality_level=hyperparameters['FSS_qualityLevel'])

print(default_points.shape)
print(special_points.shape)

# Plot the first frame with the feature points
plt.imshow(old_gray, cmap='gray')
for iny in range(default_points.shape[0]):
    plt.scatter(default_points[iny, 0, 0], default_points[iny, 0, 1], c='r')

for iny in range(special_points.shape[0]):
    plt.scatter(special_points[iny, 0, 0], special_points[iny, 0, 1], c='b')

plt.show()